In [64]:
# https://www.kaggle.com/code/ammarnassanalhajali/optimizing-the-reliability-and-cost-with-nsga-ii/notebook

In [1]:
###################################################
#-----importing library---------------------------
###################################################
import pandas as pd
import numpy as np
import time
import copy

In [6]:
df = pd.read_csv(r'/content/drive/MyDrive/Moop/wish/wish_data.csv')
df.columns

Index(['merchant_id', 'title', 'product_id', 'Unnamed: 3', 'Unnamed: 4',
       'discount', 'budget', 'units', 'normal_price', 'selling_price',
       'cost_price', 'current inventory', 'max inventory', 'units_sold',
       'baseline_sales', 'true lift', 'revenue (maximised)',
       'lift (maximised)', 'ending inventory (minimised)',
       'profit (maximise)'],
      dtype='object')

In [7]:
num_task=100           # number of Tasks
num_vm=40              # number of virtual machines
#-----Genetic Algorithm Paramaters ----------------
population_size= 100
num_iteration=500
crossover_rate= 0.8
mutation_rate= 0.3
mutation_selection_rate= 0.4
num_mutation_jobs=round(num_task*mutation_selection_rate)
start_time = time.time()

In [ ]:
Tasks_Vms_Cost = df[['merchant_id', 'product_id', 'selling_price', 'cost_price', 'budget','current inventory', 'max inventory', 'units_sold', 'baseline_sales', 'revenue (maximised)','lift (maximised)',
                     'ending inventory (minimised)', 'profit (maximise)']]
Tasks_Vms_Cost['budget'] = 10000

In [ ]:
sp = df.pivot_table(index=['merchant_id'], columns = ['product_id'], values=['selling_price'],  aggfunc=lambda x: x).fillna(0)
cp = df.pivot_table(index=['merchant_id'], columns = ['product_id'], values=['cost_price'],  aggfunc=lambda x: x).fillna(0)
bd = df.pivot_table(index=['merchant_id'], columns = ['product_id'], values=['budget'],  aggfunc=lambda x: x).fillna(0)
#ci = df.pivot_table(index=['merchant_id'], columns = ['product_id'], values=['current inventory'],  aggfunc=lambda x: x).fillna(0)
mi = df.pivot_table(index=['merchant_id'], columns = ['product_id'], values=['max inventory'],  aggfunc=lambda x: x).fillna(0)
us = df.pivot_table(index=['merchant_id'], columns = ['product_id'], values=['units_sold'],  aggfunc=lambda x: x).fillna(0)
bs = df.pivot_table(index=['merchant_id'], columns = ['product_id'], values=['baseline_sales'],  aggfunc=lambda x: x).fillna(0)
rev = df.pivot_table(index=['merchant_id'], columns = ['product_id'], values=['revenue (maximised)'],  aggfunc=lambda x: x).fillna(0)
lift = df.pivot_table(index=['merchant_id'], columns = ['product_id'], values=['lift (maximised)'],  aggfunc=lambda x: x).fillna(0)
inv = df.pivot_table(index=['merchant_id'], columns = ['product_id'], values=['ending inventory (minimised)'],  aggfunc=lambda x: x).fillna(0)
prof = df.pivot_table(index=['merchant_id'], columns = ['product_id'], values=['profit (maximise)'],  aggfunc=lambda x: x).fillna(0)

In [41]:
for i in range(len(sp_n)):
  for j in range(len(sp_n[i])):
    if (sp_n[i][j] == np.array([1.85, 1.85])).any():
      print(i, j)

2 759


In [55]:
sp.T['5268759b736046671957190c'][759][0]

1.85

In [10]:
sp_ar = np.array(sp)
cp_ar = np.array(cp)

In [11]:
sp = pd.DataFrame(sp_ar)
cp = pd.DataFrame(cp_ar)

In [26]:
# speed up the data search
sp_n=[list(map(np.float_, sp.iloc[i])) for i in range(num_task)]
cp_n=[list(map(np.float_,cp.iloc[i])) for i in range(num_task)]
start_time = time.time()

In [21]:
# for every row,
# inventory min, profit max, revenue max, lift max
# y : revenue, lift, ending inventory (minimised), 'profit (maximise)
# x : rest everything

In [62]:
def non_dominated_sorting(population_size,chroms_obj_record):
    s,n={},{}
    front,rank={},{}
    front[0]=[]
    for p in range(population_size*2):
        s[p]=[]
        n[p]=0
        for q in range(population_size*2):


            if (type(chroms_obj_record[p][0])==np.ndarray):
              chroms_obj_record[p][0] = chroms_obj_record[p][0][0]

            if (type(chroms_obj_record[q][0])==np.ndarray):
              chroms_obj_record[q][0] = chroms_obj_record[q][0][0]

            if (type(chroms_obj_record[p][1])==np.ndarray):
              chroms_obj_record[p][1] = chroms_obj_record[p][1][0]

            if (type(chroms_obj_record[q][1])==np.ndarray):
              chroms_obj_record[q][1] = chroms_obj_record[q][1][0]

            print(p, chroms_obj_record[p][1], q, chroms_obj_record[q][1])
            if ((chroms_obj_record[p][0]>chroms_obj_record[q][0] and chroms_obj_record[p][1]<chroms_obj_record[q][1])
            or (chroms_obj_record[p][0]>=chroms_obj_record[q][0] and chroms_obj_record[p][1]<chroms_obj_record[q][1])
            or (chroms_obj_record[p][0]>chroms_obj_record[q][0] and chroms_obj_record[p][1]<=chroms_obj_record[q][1])):
                if q not in s[p]:
                    s[p].append(q)
            elif ((chroms_obj_record[p][0]<chroms_obj_record[q][0] and chroms_obj_record[p][1]>chroms_obj_record[q][1])
            or (chroms_obj_record[p][0]<=chroms_obj_record[q][0] and chroms_obj_record[p][1]>chroms_obj_record[q][1])
            or (chroms_obj_record[p][0]<chroms_obj_record[q][0] and chroms_obj_record[p][1]>=chroms_obj_record[q][1])):
                n[p]=n[p]+1
        if n[p]==0:
            rank[p]=0
            if p not in front[0]:
                front[0].append(p)

    i=0
    while (front[i]!=[]):
        Q=[]
        for p in front[i]:
            for q in s[p]:
                n[q]=n[q]-1
                if n[q]==0:
                    rank[q]=i+1
                    if q not in Q:
                        Q.append(q)
        i=i+1
        front[i]=Q

    del front[len(front)-1]
    return front

###################################################
#-----Calculate crowding distance function---------
###################################################

def calculate_crowding_distance(front,chroms_obj_record):

    distance={m:0 for m in front}
    for o in range(2):
        obj={m:chroms_obj_record[m][o] for m in front}
        sorted_keys=sorted(obj, key=obj.get)
        distance[sorted_keys[0]]=distance[sorted_keys[len(front)-1]]=999999999999
        for i in range(1,len(front)-1):
            if len(set(obj.values()))==1:
                distance[sorted_keys[i]]=distance[sorted_keys[i]]
            else:
                distance[sorted_keys[i]]=distance[sorted_keys[i]]+(obj[sorted_keys[i+1]]-obj[sorted_keys[i-1]])/(obj[sorted_keys[len(front)-1]]-obj[sorted_keys[0]])

    return distance

###################################################
#-----Selection------------------------------------
###################################################
def selection(population_size,front,chroms_obj_record,total_chromosome):
    N=0
    new_pop=[]
    while N < population_size:
        for i in range(len(front)):
            N=N+len(front[i])
            if N > population_size:
                distance=calculate_crowding_distance(front[i],chroms_obj_record)
                sorted_cdf=sorted(distance, key=distance.get)
                sorted_cdf.reverse()
                for j in sorted_cdf:
                    if len(new_pop)==population_size:
                        break
                    new_pop.append(j)
                break
            else:
                new_pop.extend(front[i])

    population_list=[]
    for n in new_pop:
        population_list.append(total_chromosome[n])

    return population_list,new_pop

In [63]:
best_list,best_obj=[],[]
population_list=[]
for i in range(population_size):
    #print('i=',i)
    nxm_random_num=list(np.random.permutation(num_task)) # generate a random permutation of 0 to num_job*num_mc-1
    population_list.append(nxm_random_num) # add to the population_list
    for j in range(num_task):
        population_list[i][j]=population_list[i][j]%num_vm # convert to job number format, every job appears m times


for n in range(num_iteration):
    ###################################################
    #-----Crossover------------------------------------
    ###################################################
    parent_list=copy.deepcopy(population_list)
    offspring_list=[]
    S=list(np.random.permutation(population_size)) # generate a random sequence to select the parent chromosome to crossover

    for m in range(int(population_size/2)):

        parent_1= population_list[S[2*m]][:]
        parent_2= population_list[S[2*m+1]][:]
        child_1=parent_1[:]
        child_2=parent_2[:]

        cutpoint=list(np.random.choice(num_task, 2, replace=False))
        cutpoint.sort()

        child_1[cutpoint[0]:cutpoint[1]]=parent_2[cutpoint[0]:cutpoint[1]]
        child_2[cutpoint[0]:cutpoint[1]]=parent_1[cutpoint[0]:cutpoint[1]]

        offspring_list.extend((child_1,child_2))

    ###################################################
    #-----Mutation-------------------------------------
    ###################################################
    for m in range(len(offspring_list)):
        mutation_prob=np.random.rand()
        if mutation_rate <= mutation_prob:
            m_chg=list(np.random.choice(num_task, num_mutation_jobs, replace=False)) # chooses the position to mutation
            t_value_last=offspring_list[m][m_chg[0]] # save the value which is on the first mutation position
            for i in range(num_mutation_jobs-1):
                offspring_list[m][m_chg[i]]=offspring_list[m][m_chg[i+1]] # displacement

            offspring_list[m][m_chg[num_mutation_jobs-1]]=t_value_last

    ###################################################
    #-----Fitness valuse ------------------------------
    ###################################################
    total_chromosome=copy.deepcopy(parent_list)+copy.deepcopy(offspring_list)
    chroms_obj_record={}
    #print(sp_n)
    #print("----------------------------------------\n", cp_n)
    for m in range(population_size*2):
        gen_c=0
        gen_r=1
        for nn in range(num_task):
            gen_c +=sp_n[nn][total_chromosome[m][nn]]
            gen_r *=cp_n[nn][total_chromosome[m][nn]]
        chroms_obj_record[m]=[gen_c,gen_r]

    ###################################################
    #-----Non-dominated sorting -----------------------
    ###################################################
    #print(chroms_obj_record)
    front=non_dominated_sorting(population_size,chroms_obj_record)

    ###################################################
    #-----Selection -----------------------------------
    ###################################################
    population_list,new_pop=selection(population_size,front,chroms_obj_record,total_chromosome)
    new_pop_obj=[chroms_obj_record[k] for k in new_pop]

    ###################################################
    #-----Comparison ----------------------------------
    ###################################################
    if n==0:
        best_list=copy.deepcopy(population_list)
        best_obj=copy.deepcopy(new_pop_obj)
    else:
        total_list=copy.deepcopy(population_list)+copy.deepcopy(best_list)
        total_obj=copy.deepcopy(new_pop_obj)+copy.deepcopy(best_obj)

        now_best_front=non_dominated_sorting(population_size,total_obj)
        best_list,best_pop=selection(population_size,now_best_front,total_obj,total_list)
        best_obj=[total_obj[k] for k in best_pop]
###################################################
#-----Results ------------------------------------
###################################################
print('-----Results -----------------------------')
print("One chromosome(1x100)=",best_list[0])
print("[Reliability,Cost]=",best_obj[0])
print("------------------------------------------")
print('The elapsed time:%s'% (time.time() - start_time))

Streaming output truncated to the last 5000 lines.
170 0.0 115 0.0
170 0.0 116 0.0
170 0.0 117 0.0
170 0.0 118 0.0
170 0.0 119 0.0
170 0.0 120 0.0
170 0.0 121 0.0
170 0.0 122 0.0
170 0.0 123 0.0
170 0.0 124 0.0
170 0.0 125 0.0
170 0.0 126 0.0
170 0.0 127 0.0
170 0.0 128 0.0
170 0.0 129 0.0
170 0.0 130 0.0
170 0.0 131 0.0
170 0.0 132 0.0
170 0.0 133 0.0
170 0.0 134 0.0
170 0.0 135 0.0
170 0.0 136 0.0
170 0.0 137 0.0
170 0.0 138 0.0
170 0.0 139 0.0
170 0.0 140 0.0
170 0.0 141 0.0
170 0.0 142 0.0
170 0.0 143 0.0
170 0.0 144 0.0
170 0.0 145 0.0
170 0.0 146 0.0
170 0.0 147 0.0
170 0.0 148 0.0
170 0.0 149 0.0
170 0.0 150 0.0
170 0.0 151 0.0
170 0.0 152 0.0
170 0.0 153 0.0
170 0.0 154 0.0
170 0.0 155 0.0
170 0.0 156 0.0
170 0.0 157 0.0
170 0.0 158 0.0
170 0.0 159 0.0
170 0.0 160 0.0
170 0.0 161 0.0
170 0.0 162 0.0
170 0.0 163 0.0
170 0.0 164 0.0
170 0.0 165 0.0
170 0.0 166 0.0
170 0.0 167 0.0
170 0.0 168 0.0
170 0.0 169 0.0
170 0.0 170 0.0
170 0.0 171 0.0
170 0.0 172 0.0
170 0.0 173 0.0
170 0


KeyboardInterrupt



SyntaxError: invalid syntax. Perhaps you forgot a comma? (<ipython-input-58-c49592fbb099>, line 1)